<a href="https://colab.research.google.com/github/aliciasply/Final-Project/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import os
from IPython.display import clear_output
try:
  spark_version = 'spark-3.1.1'
  os.environ['SPARK_VERSION']=spark_version

  # Install Spark and Java
  !apt-get update
  !apt-get install openjdk-11-jdk-headless -qq > /dev/null
  !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
  !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
  !pip install -q findspark

  # Set Environment Variables
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
  os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
except:
  print(f"Error installing {spark_version}")
finally:
  clear_output()
  print(f'{spark_version} successfully installed')

# Start a SparkSession
import findspark
findspark.init()

spark-3.1.1 successfully installed


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [74]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [75]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://fakenewsdetector.s3.us-east-2.amazonaws.com/new_test_data.csv"
spark.sparkContext.addFile(url)
test_df = spark.read.csv(SparkFiles.get("new_test_data.csv"), sep=",", header=True)
test_df.columns
# df2 = test_df.select(split(col("id\ttitle\ttext\tlabel"),"\t"))

['id', 'title', 'text', 'label']

In [76]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="new_test_data.csv"
spark.sparkContext.addFile(url)
test_df = spark.read.csv(SparkFiles.get("new_test_data.csv"), sep=",", header=True)
test_df.columns
# df2 = test_df.select(split(col("id\ttitle\ttext\tlabel"),"\t"))

['id', 'title', 'text', 'label']

In [85]:
# Read data
from pyspark import SparkFiles
url ="new_test_data2.json"
spark.sparkContext.addFile(url)
test_df = spark.read.json(SparkFiles.get("new_test_data2.json"))
test_df.columns

['id', 'label', 'text', 'title']

In [90]:
test_df.show()

+-----+-----+--------------------+--------------------+
|   id|label|                text|               title|
+-----+-----+--------------------+--------------------+
| 8476| FAKE|Daniel Greenfield...|You Can Smell Hil...|
|10294| FAKE|Google Pinterest ...|Watch The Exact M...|
| 3608| REAL|U.S. Secretary of...|Kerry to go to Pa...|
|10142| FAKE|— Kaydee King (@K...|Bernie supporters...|
|  875| REAL|It's primary day ...|The Battle of New...|
| 6903| FAKE|  
I’m not an imm...|         Tehran, USA|
| 7341| FAKE|Share This Baylee...|Girl Horrified At...|
|   95| REAL|A Czech stockbrok...|‘Britain’s Schind...|
| 4869| REAL|Hillary Clinton a...|Fact check: Trump...|
| 2909| REAL|Iranian negotiato...|Iran reportedly m...|
| 1357| REAL|CEDAR RAPIDS, Iow...|With all three Cl...|
|  988| REAL|Donald Trump’s or...|Donald Trump’s Sh...|
| 7041| FAKE|Click Here To Lea...|Strong Solar Stor...|
| 7623| FAKE|October 31, 2016 ...|10 Ways America I...|
| 1571| REAL|Killing Obama adm...|Trump takes on

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://fakenewsdetector.s3.us-east-2.amazonaws.com/shapiro_cenk.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("shapiro_cenk.csv"), sep=",", header=True)

In [ ]:
df.show(5)

+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+--------------------+-------+---------+--------------------+--------+
|            tweet_id|                Time|     Ticks|            TweetUrl|               tweet|UserHandle|   UserName|          RetweetNum|LikeNum|   UserID|             UserUrl|Location|
+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+--------------------+-------+---------+--------------------+--------+
| 1374091474220949506|Mon Mar 22 20:11:...|1616443911|https://twitter.c...|"""They seem to h...|benshapiro|Ben Shapiro|                 176|    868|289548939|https://twitter.c...|    null|
| 1374173647150125060|Tue Mar 23 01:38:...|1616463503|https://twitter.c...|https://t.co/KJ25...|benshapiro|Ben Shapiro|                 220|   2266|179245596|https://twitter.c...|    null|
| 1374170856893652994|Tue Mar 23 01:27:...|1616462837|h

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer